### Notes for Modification

- Wake - Hands overlapping with face.
- Goose - Multiple variations of signs to represent the same thing.
- Nap - Hands overlapping with face/ Eyes close
- Give - Hands are closed.
- After - Hands overlapping
- Mouth - Hand closed, hand over mouth.

Commons Things:
- Closed hands
- Hands over face
- Overlapping landmarks

Solutions:
- Include eye landmarks.
- Backfill missing data with previous landmark data.
- Preserve relative distances between landmarks.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

from layers.PreprocessLayer import PreprocessLayer
from utils.Utils import print_shape_dtype, pd_read_s3_parquet, upload_file 

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import scipy
import boto3
import io
import wandb
import json

2023-04-10 18:30:01.732606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 18:30:03.312750: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow versi

In [2]:
tf.get_logger().setLevel('INFO')

In [6]:
with open("./config/config.json") as fp:
    config = json.load(fp)

In [7]:
s3_client = boto3.client(
    "s3"
)

In [8]:
AWS_S3_BUCKET = "w251-asl-data"
TRAIN_CSV_FILE = "raw-data/train.csv"

In [9]:
train_file = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=TRAIN_CSV_FILE)

In [10]:
train = pd.read_csv(train_file.get("Body"))

N_SAMPLES = len(train)
print(f'N_SAMPLES: {N_SAMPLES}')

N_SAMPLES: 94477


In [11]:
# Get complete file path to file
def get_file_path(path):
    return f'{AWS_S3_BUCKET}/raw-data/{path}'

train['file_path'] = train['path'].apply(get_file_path)

In [12]:
# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [13]:
display(train.head(30))
display(train.info())

,path,participant_id,sequence_id,sign,file_path,sign_ord
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,w251-asl-data/raw-data/train_landmark_files/26...,25
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,w251-asl-data/raw-data/train_landmark_files/28...,232
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,w251-asl-data/raw-data/train_landmark_files/16...,48
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,w251-asl-data/raw-data/train_landmark_files/25...,23
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,w251-asl-data/raw-data/train_landmark_files/62...,164
5,train_landmark_files/26734/1000241583.parquet,26734,1000241583,duck,w251-asl-data/raw-data/train_landmark_files/26...,67
6,train_landmark_files/26734/1000255522.parquet,26734,1000255522,minemy,w251-asl-data/raw-data/train_landmark_files/26...,143
7,train_landmark_files/32319/1000278229.parquet,32319,1000278229,lips,w251-asl-data/raw-data/train_landmark_files/32...,134
8,train_landmark_files/37055/100035691.parquet,37055,100035691,flower,w251-asl-data/raw-data/train_landmark_files/37...,86
9,train_landmark_files/29302/100039661.parquet,29302,100039661,time,w251-asl-data/raw-data/train_landmark_files/29...,220


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94477 entries, 0 to 94476
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            94477 non-null  object
 1   participant_id  94477 non-null  int64 
 2   sequence_id     94477 non-null  int64 
 3   sign            94477 non-null  object
 4   file_path       94477 non-null  object
 5   sign_ord        94477 non-null  int16 
dtypes: int16(1), int64(2), object(3)
memory usage: 3.8+ MB


None

In [14]:
# Source: https://www.kaggle.com/competitions/asl-signs/overview/evaluation
ROWS_PER_FRAME = 543  # number of landmarks per frame
#w251-asl-data/raw-data/train_landmark_files/28656/3311214787.parquet

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y']
    data = pd_read_s3_parquet(pq_path[14:], AWS_S3_BUCKET, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [15]:
"""
    face: 0:468
    left_hand: 468:489
    pose: 489:522
    right_hand: 522:544
        
"""

# USE_TYPES = ['left_hand', 'pose', 'right_hand']
# START_IDX = 468
# LIPS_IDXS0 = np.array([
#         61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
#         291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
#         78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
#         95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
#     ])
# # Landmark indices in original data
# LEFT_HAND_IDXS0 = np.arange(468,489)
# RIGHT_HAND_IDXS0 = np.arange(522,543)
# POSE_IDXS0 = np.arange(502, 512)
# LANDMARK_IDXS0 = np.concatenate((LIPS_IDXS0, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, POSE_IDXS0))

LIPS_IDXS0 = [0, 11, 12, 13, 14, 15, 17, 37, 38, 39, 40, 41, 42, 61, 62, 72, 73, 
        74, 76, 77, 78, 80, 81, 82, 84, 86, 87, 88, 89, 90, 91, 95, 96, 146, 
        178, 179, 180, 181, 183, 184, 185, 191, 267, 268, 269, 270, 271, 272, 
        291, 292, 302, 303, 304, 306, 307, 308, 310, 311, 312, 314, 316, 317, 
        318, 319, 320, 321, 324, 325, 375, 402, 403, 404, 405, 407, 408, 409, 415]

EYES_IDXS0 = [  6,   7,  22,  23,  24,  25,  26,  30,  31,  33,  56, 110, 112,
       113, 122, 128, 130, 133, 144, 145, 153, 154, 155, 157, 158, 159,
       160, 161, 163, 168, 173, 188, 189, 190, 193, 196, 197, 232, 233,
       243, 244, 245, 246, 247, 249, 252, 253, 254, 255, 256, 259, 260,
       263, 286, 339, 341, 351, 357, 359, 362, 373, 374, 380, 381, 382,
       384, 385, 386, 387, 388, 390, 398, 412, 413, 414, 417, 419, 453,
       463, 464, 465, 466, 467]

POSE_IDXS0 = np.arange(489, 514)
LEFT_HAND_IDXS0 = np.arange(468,489)
RIGHT_HAND_IDXS0 = np.arange(522,543)

LANDMARK_IDXS0 = np.concatenate((LIPS_IDXS0, LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0, EYES_IDXS0, POSE_IDXS0))
N_COLS = LANDMARK_IDXS0.size

HAND_IDXS0 = np.concatenate((LEFT_HAND_IDXS0, RIGHT_HAND_IDXS0), axis=0)

# Landmark indices in processed data
# LIPS_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, LIPS_IDXS0)).squeeze()
# LEFT_HAND_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, LEFT_HAND_IDXS0)).squeeze()
# RIGHT_HAND_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, RIGHT_HAND_IDXS0)).squeeze()
# HAND_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, HAND_IDXS0)).squeeze()
# POSE_IDXS = np.argwhere(np.isin(LANDMARK_IDXS0, POSE_IDXS0)).squeeze()

# print(f'# HAND_IDXS: {len(HAND_IDXS0)}, N_COLS: {N_COLS}')
# LIPS_START = 0
# LEFT_HAND_START = LIPS_IDXS.size
# RIGHT_HAND_START = LEFT_HAND_START + LEFT_HAND_IDXS.size
# POSE_START = RIGHT_HAND_START + RIGHT_HAND_IDXS.size

# print(f'LIPS_START: {LIPS_START}, LEFT_HAND_START: {LEFT_HAND_START}, RIGHT_HAND_START: {RIGHT_HAND_START}, POSE_START: {POSE_START}')

In [17]:
preprocess_layer = PreprocessLayer(config["INPUT_SIZE"])

In [18]:
"""
    face: 0:468
    left_hand: 468:489
    pose: 489:522
    right_hand: 522:544
        
"""
def get_data(file_path):
    # Load Raw Data
    data = load_relevant_data_subset(file_path)
    # Process Data Using Tensorflow
    data = preprocess_layer(data)
    
    return data

In [19]:
version = config["DATA_VERSION"]

# Get the full dataset
def preprocess_dataset():
    # Create arrays to save data
    X = np.zeros([N_SAMPLES, config["INPUT_SIZE"], config["N_COLS"] * config["N_DIMS"]], dtype=np.float32)
    y = np.zeros([N_SAMPLES], dtype=np.int32)
    NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, config["INPUT_SIZE"]], -1, dtype=np.float32)

    for row_idx, (file_path, sign_ord) in enumerate(tqdm(train[['file_path', 'sign_ord']].values)):
        if row_idx % 5000 == 0:
            print(f'Generated {row_idx}/{N_SAMPLES}')

        data, non_empty_frame_idxs = get_data(file_path)
        X[row_idx] = data
        y[row_idx] = sign_ord
        NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        if np.isnan(data).sum() > 0:
            print(row_idx)
            return data
    
    # Save X/y
    np.save('X.npy', X)
    np.save('y.npy', y)
    np.save('NON_EMPTY_FRAME_IDXS.npy', NON_EMPTY_FRAME_IDXS)
    
    # Put to S3
    upload_file("./X.npy", AWS_S3_BUCKET, f'processed-data/v{version}/X.npy')
    upload_file("./y.npy", AWS_S3_BUCKET, f'processed-data/v{version}/y.npy')
    upload_file("./NON_EMPTY_FRAME_IDXS.npy", AWS_S3_BUCKET, f'processed-data/v{version}/NON_EMPTY_FRAME_IDXS.npy')
    
    return X, y, NON_EMPTY_FRAME_IDXS

In [20]:
X, y, NON_EMPTY_FRAME_IDXS = preprocess_dataset()

print_shape_dtype([X, y, NON_EMPTY_FRAME_IDXS], ['X', 'y', 'NON_EMPTY_FRAME_IDXS'])
print(f'# NaN Values X: {np.isnan(X).sum()}')

  0%|          | 0/94477 [00:00<?, ?it/s]

Generated 0/94477
Generated 5000/94477
Generated 10000/94477
Generated 15000/94477
Generated 20000/94477
Generated 25000/94477
Generated 30000/94477
Generated 35000/94477
Generated 40000/94477
Generated 45000/94477
Generated 50000/94477
Generated 55000/94477
Generated 60000/94477
Generated 70000/94477
Generated 75000/94477
Generated 80000/94477
Generated 85000/94477
Generated 90000/94477
X shape: (94477, 96, 454), dtype: float32
y shape: (94477,), dtype: int32
NON_EMPTY_FRAME_IDXS shape: (94477, 96), dtype: float32
# NaN Values X: 0


In [ ]:
display(pd.Series(y).value_counts().to_frame('Class Count').iloc[[0,1,2,3,4, -5,-4,-3,-2,-1]])

In [ ]:
X = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{version}/X.npy')
X = np.load(io.BytesIO(X['Body'].read()))

y = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{version}/y.npy')
y = np.load(io.BytesIO(y['Body'].read()))

NON_EMPTY_FRAME_IDXS = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=f'processed-data/v{version}/NON_EMPTY_FRAME_IDXS.npy')
NON_EMPTY_FRAME_IDXS = np.load(io.BytesIO(NON_EMPTY_FRAME_IDXS['Body'].read()))

In [ ]:
data = pd_read_s3_parquet("raw-data/train_landmark_files/28656/1000106739.parquet", AWS_S3_BUCKET)

In [ ]:
data[(data['frame']==29) & (data["type"] == "pose")]

In [ ]:
data[(data['frame']==29) & (data.index.isin(POSE_IDXS0))].plot.scatter(x='x',y='y', marker='.')